<a href="https://colab.research.google.com/github/CarolineAndradeR/Data-Science/blob/main/Processo_de_decis%C3%A3o_com_predi%C3%A7%C3%A3o__anal%C3%ADtica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conceitos**

Processo de decisão:
1. Definir o problema
* Entender a situação
* Entender os objetivos
2. Identificar as alternativas
* Discussão dos detalhes do problema
* criação das alternativas
3. Definir o modelo para o problema
* Modelo de incerteza
* Modelo de preferência
4. Classificar as melhores alternativas
* Definir no tipo de score
* Ponderar as alternativas
5. Implementar a melhor solução
* Tomar a decisão
* Comunicar aos grupos


Uma análise descritiva permite entender como os indivíduos pensam e se comportam, como percebem as incertezas como aprendem e se adaptam às situações. Esse tipo de análise requer bastante observação empírica dos envolvidos.

A análise preditiva com a finalidade de entender cenários futuros e assim, auxiliar nas tomadas de decisão.
Dado que temos um processo de decisão, podemos utilizar avóres de decisão para melhor estruturar o problema. Nessa metodologia, podemos mapear os possíveis cresultados das escolhas relacionadas. Ou seja, o analista combinará os possíveis resultados e as probabilidades desses resultados ocorrerem.


# **Prático**

In [ ]:
# gerais
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# preditiva
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
# carregar dados
dados = pd.read_csv('dados_credito_small.csv')

In [ ]:
# dados gastos
dados.head()

In [ ]:
# ajuste
dados['target'] = np.where(dados['default'] == 'pagou', 1, 0)

In [ ]:
# variaveis numericas - media
dados.groupby('default')    .agg(idade_media = pd.NamedAgg('idade', 'mean'),
          renda_media = pd.NamedAgg('renda', 'mean'),
          divida_media = pd.NamedAgg('divida', 'mean'),
          outras_div_media = pd.NamedAgg('outras_div', 'mean')).reset_index()

In [ ]:
# variaveis numericas - media
dados.groupby('default')      .agg(idade_media = pd.NamedAgg('idade', 'median'),
          renda_media = pd.NamedAgg('renda', 'median'),
          divida_media = pd.NamedAgg('divida', 'median'),
          outras_div_media = pd.NamedAgg('outras_div', 'median'))     .reset_index()

In [ ]:
# categoria educacao
dados.groupby(['default', 'educacao_cat']).agg(n = pd.NamedAgg('id', 'count')).reset_index()
# categoria faixa etária
dados.groupby(['default', 'idade_cat']).agg(n = pd.NamedAgg('id', 'count')).reset_index()

In [ ]:
# logistica statsmodels
logistic_model_sm = sm.logit(formula = 'target ~ renda + divida + outras_div + C(educacao_cat) + C(idade_cat)', data = dados).fit()

In [ ]:
# dados treino e teste
train_data, test_data = train_test_split(dados,
                                         test_size = 0.25,
                                         random_state = 6451)

In [ ]:
# variavel educacao_cat é categorica, precisamos ajustar
educacao_data_train = pd.get_dummies(data=train_data['educacao_cat'], drop_first=True)
educacao_data_test = pd.get_dummies(data=test_data['educacao_cat'], drop_first=True)
# variavel idade_cat é categorica, precisamos ajustar
idade_data_train = pd.get_dummies(data=train_data['idade_cat'], drop_first=True)
idade_data_test = pd.get_dummies(data=test_data['idade_cat'], drop_first=True)
# exemplo dados educacao
educacao_data_train.head()
# exemplo dados idade
idade_data_train.head()

In [ ]:
# treino
cols_reg = ['renda', 'divida', 'outras_div']
X = train_data[cols_reg]
X = pd.concat([X, educacao_data_train, idade_data_train], axis=1)
y = train_data['target']
# teste
cols_reg = ['renda', 'divida', 'outras_div']
X_test = test_data[cols_reg]
X_test = pd.concat([X_test, educacao_data_test, idade_data_test], axis=1)
y_test = test_data['target']

In [ ]:
# treino
model_logistic = LogisticRegression(max_iter = 1000)
model_logistic.fit(X, y)
# coeficientes
betas = np.append(model_logistic.intercept_, model_logistic.coef_)
colunas = ['intercept']+X.columns.to_list()
coeficientes = pd.DataFrame(betas, colunas, columns=['Coeficientes'])
coeficientes

In [ ]:
# previsao
y_pred = model_logistic.predict(X_test)

# matriz de confusao
classes = dados['default'].unique()
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_chart = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=classes)
conf_matrix_chart.plot()

In [ ]:
# acuracia
accuracy_score(y_test, y_pred)
# erros
falsos_positivos = conf_matrix[0][1]
falsos_negativos = conf_matrix[1][0]
# acertos
verdadeiros_positivos = conf_matrix[1][1]
# custos
nao_pagamento = 1000
custo_cobranca = 100
recuperar_divida = 1000
# custos totais de cobrar e nao receber
deixou_receber = falsos_positivos * nao_pagamento
# custos totais de nao cobrar
deixar_receber_por_nao_cobrar = falsos_negativos * recuperar_divida
# custos operacionais da cobrança
custo_cobranca_total = custo_cobranca * (verdadeiros_positivos + falsos_positivos)
# Retornos
retorno_cobrar_correto = verdadeiros_positivos * recuperar_divida
# Lucro
lucro = retorno_cobrar_correto - (deixar_receber_por_nao_cobrar + deixou_receber + custo_cobranca_total)
lucro